In [1]:
import pandas as pd
df = pd.read_csv("Resources/Sparkling.csv")
df

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Crémant de Loire Brut 2017,France,Crémant de Loire,Langlois-Chateau,3.9,25,14.41,2017
1,Vicar's Choice Sauvignon Blanc Bubbles 2019,New Zealand,Marlborough,Saint Clair,4.2,25,11.90,2019
2,Crémant de Limoux Brut N.V.,France,Crémant de Limoux,Prima Perla,3.5,25,14.32,N.V.
3,Athesis Brut 2016,Italy,Südtirol - Alto Adige,Kettmeir,4.0,25,21.50,2016
4,Blanc de Blancs Réserve Brut Nature N.V.,Germany,Pfalz,Eymann,3.9,25,31.18,N.V.
...,...,...,...,...,...,...,...,...
1002,Special Cuvée Brut Aÿ Champagne N.V.,France,Champagne,Bollinger,4.2,37765,46.00,N.V.
1003,Brut Premier Champagne N.V.,France,Champagne Premier Cru,Louis Roederer,4.2,40004,36.48,N.V.
1004,Impérial Brut Champagne N.V.,France,Champagne,Moët & Chandon,4.1,76037,40.61,N.V.
1005,Brut (Carte Jaune) Champagne N.V.,France,Champagne,Veuve Clicquot,4.2,86839,43.60,N.V.


In [2]:
df = df[df.Year != "N.V."]
df = df.drop(columns = "NumberOfRatings")

In [12]:
df.to_csv('scraped_Sparkling.csv', index=False)

In [13]:
dummies = pd.get_dummies(df["Name"])
dummies.head()
Counrty_dummies = pd.get_dummies(df["Country"])
Region_dummies = pd.get_dummies(df["Region"])
Winery_dummies = pd.get_dummies(df["Winery"])
empty_df = df.drop(columns=["Name", "Country", "Region", "Winery"])

In [14]:
df_scaled = pd.concat([empty_df, dummies, Region_dummies, Counrty_dummies, Winery_dummies], axis =1)
df_scaled.head()

,Rating,Price,Year,'61 Franciacorta Nature 2012,'Giulio Ferrari' Riserva del Fondatore 2007,'Giulio Ferrari' Riserva del Fondatore 2008,'Giulio Ferrari' Riserva del Fondatore Rosé 2006,Alexandra Champagne Rosé (Grande Cuvée) 2004,Alta Langa Contessa Rosa Rosé 2012,Amour de Deutz Millésime Brut Champagne 2007,...,Vardon Kennett,Veuve A. Devaux,Veuve Clicquot,Vietti,Vilarnau,Villa Maria,Villa Sandi,Weingüter Wegeler,Winzervereinigung Freyburg-Unstrut,Zonin
0,3.9,14.41,2017,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.2,11.90,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.0,21.50,2016,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4.0,22.13,2015,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,3.9,57.51,2012,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [16]:
from sklearn.model_selection import train_test_split
X = df_scaled.drop(columns="Rating")
y = df_scaled["Rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
model.fit(X, y)

LinearRegression()

In [18]:
LinearRegression()

LinearRegression()

In [19]:
predicted_y_values = model.predict(X)

In [20]:
copy_df = df.copy()

In [21]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print releveant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 1.0.
The r2 is 1.0.
The mean squared error is 2.1947341034535277e-26.
The root mean squared error is 1.481463500547188e-13.
The standard deviation is 0.28456569346063876.
